## 4. Predicción de fugas de clientes en una compañía telefónica (7 puntos)

Una compañía telefónica está interesada en que desarrollemos un modelo que prediga los **100 clientes actuales** (dataset de explotaación) que tienen más probabilidad de abandonar la compañía. Para ello nos proporcionan una base de datos **fuga_clientes_empresa_telefonica_construccion.csv** con casos etiquetados, que usaremos para construir nuestro modelo de predicción.

Los campos de esta base de datos son:

* **Customer ID**

* **network_age**: antigüedad del cliente en días

* **Customer tenure in months:** antigüedad del cliente en meses

* **Total Spend in Months 1 and 2:** gasto total del cliente en los meses de referencia 1 y 2

* **Total SMS Spend:** gasto total en SMS

* **Total Data Spend:** gasto total en datos/internet

* **Total Data Consumption:** consumo total de datos (en KB) durante el período de estudio

* **Total Unique Calls:** número total de llamadas únicas

* **Total Onnet spend:** gasto total en llamadas a otros usuarios de la misma red de telefonía

* **Total Offnet spend:** gasto total en llamadas a otros usuarios de redes de telefonía diferentes

* **Total Call centre complaint calls:** número de llamadas de queja al call center

* **Network type subscription in Month 1:** suscripción de tipo de red en el mes 1. Esto indica la suscripción de red preferida de un cliente, lo que puede indicar su tipo de dispositivo: servicio 2G o 3G

* **Network type subscription in Month 2:** igual que el anterior pero en el mes posterior

* **Churn Status**: el valor es 1 si el cliente abandona la compañía telefónica, 0 si permanece en ella

* **Most Loved Competitor network in Month 1:** qué otro proveedor de la competencia prefiere el cliente en el mes 1. En realidad es un conjunto de columnas, cada una enfocada en un proveedor particular

* **Most Loved Competitor network in Month 2:** qué otro proveedor de la competencia prefiere el cliente en el mes 2. En realidad es un conjunto de columnas, cada una enfocada en un proveedor particular

La variable a predecir es **Churn Status**: el valor es 1 si el cliente **abandona** la compañía, 0 si no la abandona.

La compañía también nos proporciona otra base de datos, **fuga_clientes_empresa_telefonica_explotacion.csv**, con información sobre clientes de los que no se sabe ahora mismo si van a permanecer o no en la compañía. Por tanto en esta segunda base de datos todos los valores de la columna **Churn Status** son missing values (NaN).

La compañía nos pide que proporcionemos los IDs de los 100 clientes de la base de datos de explotación que con mayor probabilidad vayan a abandonar la compañía. Para ello proporcionaremos como entregable a la compañía un archivo csv con una sola columna, **Customer ID**, y 100 filas que contengan los IDs de los clientes seleccionados.

El fichero **ejemplo_fichero_predicciones.csv** contiene un ejemplo con el formato solicitado para este archivo.

In [177]:
import pandas as pd
import numpy as np

In [178]:
datos_construccion = pd.read_csv("./fuga_clientes_empresa_telefonica_construccion.csv")
datos_explotacion  = pd.read_csv("./fuga_clientes_empresa_telefonica_explotacion.csv")

In [179]:
datos_construccion.columns

Index(['Customer ID', 'network_age', 'Customer tenure in month',
       'Total Spend in Months 1 and 2', 'Total SMS Spend', 'Total Data Spend',
       'Total Data Consumption', 'Total Unique Calls', 'Total Onnet spend',
       'Total Offnet spend', 'Total Call centre complaint calls',
       'Churn Status', 'Most Loved Competitor network in Month 1_0',
       'Most Loved Competitor network in Month 1_Mango',
       'Most Loved Competitor network in Month 1_PQza',
       'Most Loved Competitor network in Month 1_ToCall',
       'Most Loved Competitor network in Month 1_Uxaa',
       'Most Loved Competitor network in Month 1_Weematel',
       'Most Loved Competitor network in Month 1_Zintel',
       'Most Loved Competitor network in Month 2_Mango',
       'Most Loved Competitor network in Month 2_PQza',
       'Most Loved Competitor network in Month 2_ToCall',
       'Most Loved Competitor network in Month 2_Uxaa',
       'Most Loved Competitor network in Month 2_Weematel',
       'Most 

In [180]:
datos_explotacion.columns

Index(['Customer ID', 'network_age', 'Customer tenure in month',
       'Total Spend in Months 1 and 2', 'Total SMS Spend', 'Total Data Spend',
       'Total Data Consumption', 'Total Unique Calls', 'Total Onnet spend',
       'Total Offnet spend', 'Total Call centre complaint calls',
       'Churn Status', 'Most Loved Competitor network in Month 1_0',
       'Most Loved Competitor network in Month 1_Mango',
       'Most Loved Competitor network in Month 1_PQza',
       'Most Loved Competitor network in Month 1_ToCall',
       'Most Loved Competitor network in Month 1_Uxaa',
       'Most Loved Competitor network in Month 1_Weematel',
       'Most Loved Competitor network in Month 1_Zintel',
       'Most Loved Competitor network in Month 2_Mango',
       'Most Loved Competitor network in Month 2_PQza',
       'Most Loved Competitor network in Month 2_ToCall',
       'Most Loved Competitor network in Month 2_Uxaa',
       'Most Loved Competitor network in Month 2_Weematel',
       'Most 

In [181]:
# chequeo de que las columnas son exactamente las mismas en los dos ficheros:

sum(datos_construccion.columns != datos_explotacion.columns)

0

In [182]:
# la columna a predecir es "Churn Status"
# si es 1, el cliente se va de la compañía
# si es 0, el cliente se queda

# Esta columna se sabe en el dataset de construcción (ejemplos de clientes pasados):
datos_construccion["Churn Status"].values[:10]

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1])

In [183]:
# Sin embargo no se sabe en el dataset de explotación (clientes actuales):
datos_explotacion["Churn Status"].values[:10]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

# Resolución:

Comenzamos preprando la matriz de datos para el entrenamiento, la matriz obtenida a partir del fichero es la siguiente:

In [199]:
matriz=datos_construccion.values
matriz

array([['ADF0039', 123, 4.1, ..., 0, 0, 0],
       ['ADF0041', 1316, 43.87, ..., 0, 0, 0],
       ['ADF0051', 2385, 79.5, ..., 0, 0, 0],
       ...,
       ['ADF1304', 2971, 99.03, ..., 1, 0, 0],
       ['ADF1308', 595, 19.83, ..., 1, 0, 0],
       ['ADF1313', 1419, 47.3, ..., 1, 0, 0]], dtype=object)

Verificamos los tipos de datos que hemos leido:

In [200]:
print(datos_construccion.dtypes)

Customer ID                                           object
network_age                                            int64
Customer tenure in month                             float64
Total Spend in Months 1 and 2                        float64
Total SMS Spend                                      float64
Total Data Spend                                     float64
Total Data Consumption                               float64
Total Unique Calls                                     int64
Total Onnet spend                                      int64
Total Offnet spend                                     int64
Total Call centre complaint calls                      int64
Churn Status                                           int64
Most Loved Competitor network in Month 1_0             int64
Most Loved Competitor network in Month 1_Mango         int64
Most Loved Competitor network in Month 1_PQza          int64
Most Loved Competitor network in Month 1_ToCall        int64
Most Loved Competitor ne

Generamos estadísticas básicas sobre el dataframe:

In [201]:
datos_construccion.describe()

,network_age,Customer tenure in month,Total Spend in Months 1 and 2,Total SMS Spend,Total Data Spend,Total Data Consumption,Total Unique Calls,Total Onnet spend,Total Offnet spend,Total Call centre complaint calls,...,Most Loved Competitor network in Month 1_ToCall,Most Loved Competitor network in Month 1_Uxaa,Most Loved Competitor network in Month 1_Weematel,Most Loved Competitor network in Month 1_Zintel,Most Loved Competitor network in Month 2_Mango,Most Loved Competitor network in Month 2_PQza,Most Loved Competitor network in Month 2_ToCall,Most Loved Competitor network in Month 2_Uxaa,Most Loved Competitor network in Month 2_Weematel,Most Loved Competitor network in Month 2_Zintel
count,1110.000000,1110.000000,1110.000000,1110.000000,1110.000000,1.110000e+03,1110.000000,1110.00000,1110.000000,1110.000000,...,1110.000000,1110.000000,1110.000000,1110.000000,1110.000000,1110.000000,1110.000000,1110.000000,1110.000000,1110.000000
mean,1512.038739,50.401378,835.039915,30.687090,67.162910,2.343397e+06,203.043243,6066.10000,16273.468468,1.986486,...,0.145946,0.221622,0.096396,0.109009,0.082883,0.233333,0.047748,0.569369,0.037838,0.028829
std,1272.024046,42.400681,1317.850917,58.625098,323.714105,7.343873e+06,300.257845,11969.62004,37230.646293,2.497392,...,0.353211,0.415525,0.295267,0.311791,0.275829,0.423143,0.213328,0.495388,0.190890,0.167401
min,-5.000000,-0.170000,4.910000,0.000000,0.000000,5.860000e-02,1.000000,0.00000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,381.250000,12.707500,172.196000,3.492500,1.250000,1.410266e+03,18.000000,36.00000,1111.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1243.500000,41.450000,616.341000,14.000000,11.250000,1.472994e+05,82.000000,1488.00000,4185.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,2288.000000,76.267500,1021.458300,31.637500,50.000000,1.304779e+06,251.000000,6678.00000,13327.500000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,5451.000000,181.700000,24438.830000,665.000000,8295.000000,9.945640e+07,2184.000000,146344.00000,384648.000000,49.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Preparamos los datos para modelar:

La columna del customer ID no va a proporcionar ninguna informacion al entrenamiento por lo que la quitamos y también debemos eliminar la variable Churn Status ya que es la variable a predecir.

El índice de la columna de Customer ID es 0.
El índice de la colummna de Churn Status es 11.

In [202]:
df= np.delete(matriz, [0, 11], axis=1)
print(df)

[[123 4.1 76.714 ... 0 0 0]
 [1316 43.87 98.8904 ... 0 0 0]
 [2385 79.5 372.63 ... 0 0 0]
 ...
 [2971 99.03 832.7248 ... 1 0 0]
 [595 19.83 839.5724 ... 1 0 0]
 [1419 47.3 843.0604 ... 1 0 0]]


Controlamos posibles fallos en cada columna:

1. Tenemos que controlar las filas cuya primera columna, que es antigüedad del cliente en días, es menor que 0. Ya que mediante el describe hemos visto que el valor mínimo era negativo y eso es erroneo. Lo mismo ocurre con la columna de la antigüedad del cliente en meses.

In [203]:
np.where(df[:, 0]<0)

(array([489, 915]),)

In [204]:
np.where(df[:, 1]<0)

(array([489, 915]),)

Como los datos negativos coinciden y son solo dos, además el conjunto tiene más de 1000 instancias, podemos permitirnos perder alguna fila:

In [205]:
df= np.delete(df, np.where(df[:, 0]<0), axis=0)
print(df)

[[123 4.1 76.714 ... 0 0 0]
 [1316 43.87 98.8904 ... 0 0 0]
 [2385 79.5 372.63 ... 0 0 0]
 ...
 [2971 99.03 832.7248 ... 1 0 0]
 [595 19.83 839.5724 ... 1 0 0]
 [1419 47.3 843.0604 ... 1 0 0]]


In [206]:
np.where(df[:, 1]<0)

(array([], dtype=int64),)

Estudiando los demás valores de la tabla obtenida mediante describe, no parece que haya que hacer ningún ajuste más.

Ya tendríamos la matriz de entrenamiento lista.

También tenemos que procesar la matriz del test:

In [223]:
datos_explotacion.describe()

,network_age,Customer tenure in month,Total Spend in Months 1 and 2,Total SMS Spend,Total Data Spend,Total Data Consumption,Total Unique Calls,Total Onnet spend,Total Offnet spend,Total Call centre complaint calls,...,Most Loved Competitor network in Month 1_ToCall,Most Loved Competitor network in Month 1_Uxaa,Most Loved Competitor network in Month 1_Weematel,Most Loved Competitor network in Month 1_Zintel,Most Loved Competitor network in Month 2_Mango,Most Loved Competitor network in Month 2_PQza,Most Loved Competitor network in Month 2_ToCall,Most Loved Competitor network in Month 2_Uxaa,Most Loved Competitor network in Month 2_Weematel,Most Loved Competitor network in Month 2_Zintel
count,289.000000,289.000000,289.000000,289.000000,289.000000,2.890000e+02,289.000000,289.000000,289.000000,289.000000,...,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000
mean,1211.823529,40.393806,693.848493,29.374844,24.911349,1.472105e+06,200.726644,9712.034602,13696.986159,1.678201,...,0.138408,0.269896,0.110727,0.107266,0.031142,0.169550,0.038062,0.681661,0.031142,0.048443
std,1266.602498,42.220141,952.562211,69.052163,49.853049,4.550351e+06,309.582668,28170.365737,38325.676925,1.621334,...,0.345927,0.444676,0.314338,0.309988,0.174002,0.375888,0.191679,0.466640,0.174002,0.215073
min,-2.000000,-0.070000,16.152000,0.000000,0.000000,1.172000e-01,1.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,136.000000,4.530000,132.018000,1.800000,1.250000,1.212265e+03,17.000000,186.000000,1224.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,765.000000,25.500000,517.790000,12.250000,7.500000,8.830388e+04,77.000000,2464.000000,4005.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,1877.000000,62.570000,1004.496000,28.640000,27.500000,6.968163e+05,273.000000,8316.000000,12965.000000,2.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,5363.000000,178.770000,10042.353200,873.980000,493.750000,3.714204e+07,2596.000000,381174.000000,431440.000000,16.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Vuelve a haber valores mínimos negativos en las dos primeras columnas, los identificamos y eliminamos.

In [224]:
matriz_test=datos_explotacion.values
print(matriz_test)

[['ADF1330' 888 29.6 ... 1 0 0]
 ['ADF1331' 120 4.0 ... 1 0 0]
 ['ADF1345' 3809 126.97 ... 1 0 0]
 ...
 ['ADF1967' 203 6.77 ... 0 0 1]
 ['ADF1971' 109 3.63 ... 0 0 1]
 ['ADF1982' 904 30.13 ... 0 0 1]]


In [225]:
np.where(matriz_test[:, 1]<0)

(array([52]),)

In [226]:
np.where(matriz_test[:, 2]<0)

(array([52]),)

El valor negativo de ambas columnas coincide con el índice de una misma fila:

In [227]:
matriz_test= np.delete(matriz_test, np.where(matriz_test[:, 1]<0), axis=0)
print(matriz_test)

[['ADF1330' 888 29.6 ... 1 0 0]
 ['ADF1331' 120 4.0 ... 1 0 0]
 ['ADF1345' 3809 126.97 ... 1 0 0]
 ...
 ['ADF1967' 203 6.77 ... 0 0 1]
 ['ADF1971' 109 3.63 ... 0 0 1]
 ['ADF1982' 904 30.13 ... 0 0 1]]


In [222]:
np.where(matriz_test[:, 2]<0)

(array([], dtype=int64),)

La matriz de test ya estaría lista.